## 機械学習：野球のスイング判定

* 野球のバッティングフォームを対象とする
* 良いスイングのパターンを学習させて判定できるようにしたい

### 今回用いる方法（概要）

1. 姿勢推定により身体の部位の位置を取得
2. 動画からスイング部分を切り出す（本Notebook内では取り扱わない）
3. 時系列に位置を並べてデータ化する
4. 良いパターンと良くないパターンについて学習する

### 準備：動画の処理（再生）

* OpenCV.VideoCapture
* 1フレームずつ表示

In [ ]:
import time
import math
import numpy as np
import pandas as pd
import copy
import cv2
import matplotlib.pyplot as plt

from IPython.display import display, Image, clear_output
from pprint import pprint as pp

In [ ]:
def imshow(img, width=240):
    _, buf = cv2.imencode(".jpg", img)
    display(Image(data=buf.tobytes(), width=width))
    clear_output(wait=True)

class CvCapture():
    def __init__(self, video_path):
        self.cap = cv2.VideoCapture(video_path)
        self.fps = self.cap.get(cv2.CAP_PROP_FPS)

    def read(self):
        return self.cap.read()
    
    def get_specified_frame(self, frame_number):
        self.cap.set(cv2.CAP_PROP_POS_FRAMES, frame_number - 1)
        ret, frame = self.cap.read()
        return frame

    def release(self):
        self.cap.release()

    def process_frames(self, frame_proc=lambda x: x, target_frames=None, show_frames=True):
        frame_no = 0
        while True:
            st = time.time()
            ret, frame = self.cap.read()
            frame_no += 1
            if not ret:
                break
            elif target_frames is not None and frame_no not in target_frames:
                continue
            else:
                frame = frame_proc(frame)
            
            if show_frames:
                imshow(frame)
            else:
                print('processing', frame_no)
            
            elapsed_time = time.time() - st
            if elapsed_time < 1 / self.fps:
                time.sleep(1 /self.fps - elapsed_time)
    
        self.release()

In [ ]:
# 今回使う動画
sample = 'data/sample_0.mp4'

In [ ]:
# 試しに再生
CvCapture(sample).process_frames()

### 1. 姿勢推定により身体の部位の位置を取得

1. **姿勢推定により身体の部位の位置を取得**
2. 動画からスイング部分を切り出す（本Notebook内では取り扱わない）
3. 時系列に位置を並べてデータ化する
4. 良いパターンと良くないパターンについて学習する

**OpenPoseソースを取得する**

※以下コメントアウトを外した上で実行

In [ ]:
# !git clone https://github.com/Hzzone/pytorch-openpose.git

OpenPoseに必要なライブラリのインストール

※以下コメントアウトを外した上で実行

In [ ]:
# !pip install -r pytorch-openpose/requirements.txt -t pytorch-openpose

姿勢推定モデルのダウンロード

※以下コメントアウトを外した上で実行

In [ ]:
# !wget https://www.dropbox.com/sh/7xbup2qsn7vvjxo/AABaYNMvvNVFRWqyDXl7KQUxa/body_pose_model.pth -P pytorch-openpose/model

Python実行時のパスを通す

In [ ]:
import os, sys

# Pythonのインポートパスにopenposeを追加
openpose_path = 'pytorch-openpose'
sys.path.append(openpose_path)

print(sys.path)

In [ ]:
from src import model
from src import util as op_util
from src.body import Body

__body_estimate = Body('pytorch-openpose/model/body_pose_model.pth')

def body_estimation(img):
    c, s = __body_estimate(img)
    return np.array(c), np.array(s)

OpenPose-Pythonライブラリから提供された`Body`クラスに姿勢推定モデルを渡して初期化する。  
`Body`を実行すると指定した画像の人物およびその姿勢情報を推定して返却する。  
ここでは使いやすくするため、この処理を`body_estimation`関数として定義し、以降こちらを使用して画像の姿勢推定を行う。

**姿勢推定データを見る**

例として、動画データの30フレーム目の姿勢推定結果を表示する。

In [ ]:
# 30フレーム目を取り出す
frame = CvCapture(sample).get_specified_frame(30)
# 姿勢推定
candidate, subset = body_estimation(frame)
print('Candidate サイズ', candidate.shape)
pp(candidate)
print('Subset サイズ', subset.shape)
pp(subset)

**部位データ**

OpenPoseの姿勢推定結果には`Candidate`と`Subset`があり、`body_estimation`関数から実行結果として返却される。

Candidate
* 身体部位の検出情報
* サイズは検出された個体数×18（各個体は18個の部位データがある）
* 部位データは`[X座標, Y座標, 確度, 検出ID]`
* 複数人物が居る画像では複数の部位データがあり、その場合、検出IDが順序通りの数になるとは限らない

Subset
* 個体の検出情報
* サイズは検出された人物等の個体数（各個体には18個の検出IDとスコア）
* 検出IDは検出された場合のみ0以上の整数、検出されなければ-1が入る
* SubsetのN番目のデータはCandidateのN番目のデータに対応する

以降ではこれを**部位データ**と記述する。

**部位データを元画像に描画する処理**

CandidateとSubsetで指定された部位を繋げて骨格のように表示する。
描画処理のソースコードは別ファイルに記載。

In [ ]:
def draw_bodypose(img, candidate, subset):
    return op_util.draw_bodypose(copy.deepcopy(img), candidate, subset)

試しに実行

In [ ]:
def print_estimation(d):
    c, s = body_estimation(d)
    # `draw_bodypose`は画像データを変更するため
    # 元データを保持したい場合はcopy.deepcopy等で複製したデータを渡す
    d = draw_bodypose(d, c, s)
    return d

# 動画の40～50フレームを5フレームごとに表示
CvCapture(sample).process_frames(frame_proc=print_estimation, target_frames=np.arange(40, 51, 5))

### 2. 動画からスイング部分を切り出す（本Notebook内では取り扱わない）

1. 姿勢推定により身体の部位の位置を取得
2. **動画からスイング部分を切り出す（本Notebook内では取り扱わない）**
3. 時系列に位置を並べてデータ化する
4. 良いパターンと良くないパターンについて学習する

スイング開始・終了の判定は条件が多く、動画全体からスイング部分のみ抜き出すコードは長くなるため割愛するが  
処理の概要は以下の通り。

* スイング先頭判定が真となるまで次のフレームを読み込む
* スイング終了判定が真となるまで部位データを取得する
* スイング先頭～終了までの代表値として11個のフレームを抽出する
* 抽出した部位データを保存する（※スイング判定完成時は、ここで保存のかわりに判定を行う）
* 動画が終了するまで上記の処理を続けて行う
* 参考：スイング先頭判定：部位データを用いる
    * 手首が近い
    * 手首の高さが頭に近い
    * 手首が一定以上の位置にある
    * 腰の左右が離れている（腰がひねられていない）
    * 全身が映っている

### 3. 時系列に位置を並べてデータ化する

1. 姿勢推定により身体の部位の位置を取得
2. 動画からスイング部分を切り出す（本Notebook内では取り扱わない）
3. **時系列に位置を並べてデータ化する**
4. 良いパターンと良くないパターンについて学習する

**スイングデータ**

スイング判定モデルの学習に使われる入力データは、部位データを時系列に並べたものとなる。  
時系列での保存は動画データのフレームを順次姿勢推定する事で行う。

そこで取得データを姿勢推定して保存する処理を定義する。（推定後に描画しているが高速化のためには描画はない方が良い。）

まず、実行短縮のためフレーム画像をデータとして保存しておく。

In [ ]:
sample_frames = []

cap = CvCapture(sample)
for i in range(100):
    print('.', end='')
    frame = cap.get_specified_frame(i)
    sample_frames.append(frame)

print(len(sample_frames), 'frames')

**スイング部分の抽出**

今回はスイング開始終了を判定する箇所は作らないので、手作業で抽出フレーム番号を調整。  
以下のうちから姿勢推定が出来た11フレームのランドマークを抽出する。
* 29～40: 振りはじめ付近
* 41～52: スイング中～終了

姿勢推定が出来たかどうかはsubsetのサイズを見る。(検出できていなければ0)

In [ ]:
tmp_candidates = []
tmp_subsets = []
swing_indexes = np.arange(29, 41, 5).tolist() + np.arange(41, 53, 1).tolist()
print(swing_indexes)

# 動画から指定したフレームのみ抜き出し、描画すると共にスイングデータとして収集する
# CvCapture(sample).process_frames(frame_proc=swing.collect_data, target_frames=swing_indexes)
for frame_no in swing_indexes:
    d = sample_frames[frame_no]
    c, s = body_estimation(d)
    if s.size == 0: continue
    # print(frame_no, '.', end='')
    imshow(draw_bodypose(d, c, s))
    tmp_candidates.append(c)
    tmp_subsets.append(s)

print(' collect candidates', len(tmp_candidates))

In [ ]:
# スイングのフレーム数を11に調整する
candidates = tmp_candidates[1:-3]
subsets = tmp_subsets[1:-3]

print(len(candidates))

**正確に検出できた部位のみ抽出する**

Candidateには18個の部位通りにデータが入っているとは限らず、余分に部位が検出されたり、見えない部分が検出できなかったりする。  
以下の例は、Candidateに20個の部位が検出されているが、そのうち2つ(ID=5,17)がSubset側に入っていない(この人物として検出できていない)。

In [ ]:
print(candidates[0].shape, candidates[0])
print(subsets[0])

In [ ]:
# CandidateからSubsetに指定されているデータだけ抽出
swing_xy = []
for i, subs in enumerate(subsets):
    x = subs[0, :18].astype(int)
    swing_xy.append(candidates[i][x, :2])

swing_xy = np.array(swing_xy)
print(swing_xy.shape)
print(swing_xy[0])

11フレーム18部位の座標データとなった。

**部位データ**

OpenPoseでは18個の部位を取得するがそのうちスイングで使用する値のみ抽出する。

In [ ]:
# 部位グループ
lg_head = [0] # 鼻
lg_upper = [1, 2, 5]  # 中心、右肩、左肩
lg_arms = [3, 4, 6, 7]    # 右ひじ、右手首、左ひじ、左手首
lg_trunk = [0, 1, 8, 11]  # 鼻、中心、右腰、左腰
lg_leg = [9, 10, 12, 13]  # 右ひざ、右足首、左ひざ、左足首
# フォーム評価で用いるすべての部位
lg_all_ff = sorted(set(lg_head + lg_upper + lg_arms + lg_trunk + lg_leg))
print(lg_all_ff)

In [ ]:
# candidateから使う部分だけ取得する例
# 先頭フレームの14か所の座標を表示
print(swing_xy[0][lg_all_ff].shape)
swing_xy[0][lg_all_ff]

**データの正規化**

ここまでに得られたスイングの部位座標は画像の左上から横方向と縦方向の位置（ピクセル数）となる。  
これには以下の問題がある。
* フレーム内における人物の位置や僅かな向きの違いにより値が変わる
* 画像サイズにより値が変わる

学習データとして使いやすくするため、今回はいくつかの処理により部位座標データを0から1の間に正規化する。  

* 人物の位置により値が変わる　⇒ピクセル数の絶対値ではなく、1フレーム目からの変化にする
* 向きや画像サイズにより値が変わる　⇒変化率を用いる

以下に計算の一部を記述する。

In [ ]:
def to_train_data(data):
    # フォーム判定で使用する箇所を抽出
    s_data = copy.deepcopy(data)
    # 基準位置を0として基準位置からの移動距離に直す
    # 先頭データは基準位置(すべて0)なので除外
    s_data = s_data[1:] - s_data[0, :]
    # 移動距離を、身体の縦サイズとの移動比率に変換
    s_data = s_data / 100
    # 検出されなかった部位の値NaNを補間する
    # 間のNaNは中間値で、端のNaNは隣接値で埋める
    return s_data.reshape([s_data.shape[0], math.prod(s_data.shape[1:])])

sample_swing_td = to_train_data(swing_xy[:, lg_all_ff])
print(sample_swing_td.shape)
print(sample_swing_td)

これで1つのスイングデータ(11フレーム、14個の座標)が、機械学習用の入力データ(10フレーム、14*2=28個の変化率)となった。  
この場合、1つのスイングデータのサイズは**280**となる

### 4. 良いパターンと良くないパターンについて学習する

1. 姿勢推定により身体の部位の位置を取得
2. 動画からスイング部分を切り出す（本Notebook内では取り扱わない）
3. 時系列に位置を並べてデータ化する
4. **良いパターンと良くないパターンについて学習する**

3で正規化したスイングデータを多く用意し、そのデータからパターンを学習する。  
ここは以下の根拠から回帰分析によりスイングを学習する。

* 良いスイングと良くないスイングは一定軌道に近づく(と仮定する)
* 良いスイングと良くないスイングのパターンは異なる(と仮定する)

In [ ]:
# 良くないと仮定したスイングのプロット(頭部分の横方向移動のみ)
d = sample_swing_td
plt.scatter(np.arange(d.shape[0]), d[:, 0])

**良いスイングパターンを求める**

上のようなフレーム毎の位置変化は、良いスイングと良くないスイングとで統計的に見て異なると考えられる。  

そこで、様々なスイングデータを集め、良い=1、良くない=0として、ラベルを付ける。  
これを1行1スイングデータとしてCSVファイルとしてまとめる。

CSVレイアウトは以下の様になる。

**列番号: 項目名**
* 0: 1フレーム目の頭x(の位置変化率)
* 1: 1フレーム目の頭y
* 2: 1フレーム目の中心x
* ...
* 279: 10フレーム目の左足首y
* 280: 良いスイングである(1または0)

In [ ]:
input_data = pd.read_csv('data/train.csv', header=None)
input_data.head()

281列目に0または1があり、これがその行のスイングが良いか悪いかのラベルとなる。  
`train.csv`には、以下の通り、良いスイングのデータも含まれている。

In [ ]:
input_data.loc[input_data[280] == 1].head()

参考：OpenPose部位

~~~
０：頭(鼻)、１：中心(心臓付近)、２：右肩、3：右肘、４：右手首、５：左肩、６：左肘
７：左手首、８：右腰、９：右膝、10：右足首、11：左腰、12：左膝、13：左足首
14：右目、15：左目、16：右耳、17：左耳
~~~

試しにXGBoostによる回帰を行う

In [ ]:
import xgboost

print(xgboost.__version__)
model = xgboost.XGBRegressor(n_estimators=500, max_depth=7, eta=0.1, subsample=0.7, colsample_bytree=0.8)

print(input_data.shape)
x, y = input_data.values[:, :-1], input_data.values[:, -1]
model.fit(x, y)

先ほどの自分のスイングデータ(学習CSVデータに含まれていない)を判定してみる

In [ ]:
d = sample_swing_td.astype(float)
print(d.shape)
d = d.reshape(1, 280)
print(d.shape)
model.predict(d)

値は0から1の値を取る。  
これを「良いスイング」である確度と考えると、例えば、0.5以上であれば良いスイングと判定する事になる。